In [ ]:
import pandas as pd
import time
import requests

### Pull all pitchers' height and weight from Baseball Reference
Note: need to use vpn to avoid HTTP 429 error

In [ ]:
teams = [
    "ARI", "ATL", "BAL", "BOS", "CHC", "CHW", "CIN", "CLE",
    "COL", "DET", "HOU", "KCR", "LAA", "LAD", "MIA", "MIL",
    "MIN", "NYM", "NYY", "OAK", "PHI", "PIT", "SDP", "SEA",
    "SFG", "STL", "TBR", "TEX", "TOR", "WSN"
]
year = 2025

pitchers = None

for team in teams:
    url = f"https://www.baseball-reference.com/teams/{team}/{year}-roster.shtml"
    tables = pd.read_html(url)

    # First table on roster page
    roster = tables[0]

    team_pitchers = roster[roster['Unnamed: 4'] == 'Pitcher'][['Name', 'Ht', 'Wt']]
    # Keep only pitchers
    pitchers = team_pitchers if pitchers is None else pd.concat([pitchers, team_pitchers], ignore_index=True)
    time.sleep(1)

# Keep Name, Height, Weight
print(pitchers)
pitchers.to_csv("pitchers.csv")

In [ ]:
pitchers = pd.read_csv("pitchers.csv")

def height_to_inches(height_str):
    feet, inches = height_str.split("'")
    inches = inches.replace('"', '').strip()
    return int(feet) * 12 + int(inches)

pitchers['height_inches'] = pitchers['Ht'].apply(height_to_inches)
pitchers['height_zscore'] = (pitchers['height_inches'] - pitchers['height_inches'].mean()) / pitchers['height_inches'].std()
pitchers['weight_zscore'] = (pitchers['Wt'] - pitchers['Wt'].mean()) / pitchers['Wt'].std()

# pitchers_sorted = pitchers.sort_values(by='height_zscore', ascending=False)

# print(pitchers_sorted)

pitchers_sorted = pitchers.sort_values(by='weight_zscore', ascending=False)

print(pitchers_sorted)
